In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset_path = "Car_Prices.csv"

# using try and except to handle potential errors
try:
    chunk_size = 10000    # Processing in chunkcs which is more memory efficient for large files
    bmw_chunks = []       # an empty list to store chuncks containing BMW data 
    
    column_names = [
        'year', 'make', 'model', 'trim', 'body', 'transmission', 
        'vin', 'state', 'condition', 'odometer', 'color', 'interior',
        'seller', 'mmr', 'selling_price', 'sale_date'
    ]
    
    for chunk in pd.read_csv(
        dataset_path, 
        chunksize=chunk_size,
        header=None,
        names=column_names,
        on_bad_lines='skip',    # skipping malformed rows instead of crashing
        engine='python',        # python parser
        quoting=3,              # dont treat quotes specially
        quotechar=None,         # no quote character
        escapechar='\\'         # uses backslash as escape character
    ):
        # Filter for BMW rows in current chunck
        # adds BMW rows to the list we just created
        if 'make' in chunk.columns:
            bmw_chunk = chunk[chunk['make'].str.lower() == 'bmw']
            bmw_chunks.append(bmw_chunk)
        
        print(f"Processed chunk, current BMW count: {sum(len(c) for c in bmw_chunks)}")
    
    # if BMW data is found combine them all into one dataframe
    if bmw_chunks:
        bmw_df = pd.concat(bmw_chunks)
        bmw_df.to_csv('Uncleaned_Bmw_Data.csv', index=False)
        print(f"Extracted {len(bmw_df)} BMW rows")
    else:
        print("No BMW data found")

except Exception as e:
    print(f"Error: {e}")

Processed chunk, current BMW count: 844
Processed chunk, current BMW count: 1220
Processed chunk, current BMW count: 1459
Processed chunk, current BMW count: 1699
Processed chunk, current BMW count: 2044
Processed chunk, current BMW count: 2266
Processed chunk, current BMW count: 2683
Processed chunk, current BMW count: 3042
Processed chunk, current BMW count: 3323
Processed chunk, current BMW count: 3864
Processed chunk, current BMW count: 4544
Processed chunk, current BMW count: 4788
Processed chunk, current BMW count: 5071
Processed chunk, current BMW count: 5366
Processed chunk, current BMW count: 5973
Processed chunk, current BMW count: 6149
Processed chunk, current BMW count: 6426
Processed chunk, current BMW count: 6740
Processed chunk, current BMW count: 7179
Processed chunk, current BMW count: 7657
Processed chunk, current BMW count: 8120
Processed chunk, current BMW count: 8550
Processed chunk, current BMW count: 8750
Processed chunk, current BMW count: 8940
Processed chunk, 

In [3]:
real_dataset_path = "Uncleaned_Bmw_Data.csv"

df = pd.read_csv(real_dataset_path)
df.head(5)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,selling_price,sale_date
0,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,4.5,1331.0,gray,black,"""financial services remarketing (lease)""",31900,30000,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
1,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,4.3,2641.0,gray,black,"""financial services remarketing (lease)""",66000,67000,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,M5,Base,Sedan,automatic,wbsfv9c51ed593089,ca,3.4,14943.0,black,black,"""the hertz corporation""",69000,65000,Wed Dec 17 2014 12:30:00 GMT-0800 (PST)
3,2014,BMW,6 Series,650i,Convertible,automatic,wbayp9c53ed169260,ca,3.4,8819.0,black,black,"""the hertz corporation""",68000,67200,Wed Dec 17 2014 12:30:00 GMT-0800 (PST)
4,2014,BMW,5 Series,528i,Sedan,automatic,wba5a5c51ed501631,ca,2.9,25969.0,black,black,"""financial services remarketing (lease)""",34200,30000,Tue Feb 03 2015 04:30:00 GMT-0800 (PST)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           20640 non-null  int64  
 1   make           20640 non-null  object 
 2   model          20554 non-null  object 
 3   trim           20637 non-null  object 
 4   body           20566 non-null  object 
 5   transmission   17949 non-null  object 
 6   vin            20640 non-null  object 
 7   state          20640 non-null  object 
 8   condition      20231 non-null  float64
 9   odometer       20637 non-null  float64
 10  color          20547 non-null  object 
 11  interior       20547 non-null  object 
 12  seller         20640 non-null  object 
 13  mmr            20640 non-null  int64  
 14  selling_price  20640 non-null  int64  
 15  sale_date      20640 non-null  object 
dtypes: float64(2), int64(3), object(11)
memory usage: 2.5+ MB


In [5]:
# dropping columns we don't need
df = df.drop(columns=['make', 'vin'], axis=1)

In [6]:
# getting  unique values in this column for later mapping
print("before cleaning: ", df['transmission'].unique())

# filling NaN values
df['transmission'] = df['transmission'].fillna('Unknown')

# map each value to a number and covert it to uint8 to make it more memory efficient
df['transmission'] = df['transmission'].map({'automatic':0, 'manual':2, 'Unknown':1})
df['transmission'] = df['transmission'].astype('uint8')
df.head()


print("\nAfter mapping:")
transmission_counts = df['transmission'].value_counts()
print(transmission_counts)

before cleaning:  ['automatic' nan 'manual']

After mapping:
transmission
0    17119
1     2691
2      830
Name: count, dtype: int64


In [7]:
# converting to categorical type which is good for plotting
df['state'] = df['state'].astype('category')
df['seller'] = df['seller'].astype('category')

In [8]:
# calculating condition median and filling NaN values with it
condition_median = df['condition'].median()
df['condition'] = df['condition'].fillna(condition_median)

print(f"fillen condition NaN values with: {condition_median}")

fillen condition NaN values with: 3.8


In [9]:
# calculating odometer median and filling NaN values with it
odometer_median = df['odometer'].median()
df['odometer'] = df['odometer'].fillna(odometer_median)

print(f"filled odometer NaN values with: {odometer_median}")

filled odometer NaN values with: 51093.0


In [10]:
# For categorical columns, fill with 'unknown'
df['body'] = df['body'].fillna('unknown')
df['trim'] = df['trim'].fillna('unknown')
df['model'] = df['model'].fillna('Unknown')

In [11]:
print("unique values in 'body' column before cleaning: \n", df['body'].unique())

# converting to lower space and removing and leading or trailing whitespaces
df['body'] = df['body'].str.lower().str.strip()

# label encode the body column and convert to uint8 to save memory
le = LabelEncoder()
df['body'] = le.fit_transform(df['body']).astype('uint8')

# printing out the mapping we just did with label encoding
print("\nMapping:")
for i, label in enumerate(le.classes_):
    print(f"{label}: {i}")

unique values in 'body' column before cleaning: 
 ['Sedan' 'Convertible' 'SUV' 'Coupe' 'unknown' 'Hatchback' 'Wagon' 'sedan'
 'suv' 'convertible' 'coupe' 'wagon' 'hatchback']

Mapping:
convertible: 0
coupe: 1
hatchback: 2
sedan: 3
suv: 4
unknown: 5
wagon: 6


In [12]:
# wanted to drop rows where both color and interior is missing
# since such rows doesnt exist we need to handle missing data with a different method
""" mask = df['color'].isna() % df['interior'].isna() 
rows_to_drop = mask.sum()
print(rows_to_drop) """


print("unique values in 'color' column before cleaning \n", df['color'].unique())
print("\n\nunique values in 'interior' column before cleaning \n", df['interior'].unique())

# filling with most common
cols = ['color', 'interior']

for col in cols:
    most_common = df[col].mode()[0]

    df[col] = df[col].replace('—', pd.NA)
    df[col] = df[col].fillna(most_common)

    print(f"\nfilled {col} with: {most_common}")



le = LabelEncoder()
df['color'] = le.fit_transform(df['color']).astype('uint8')
df['interior'] = le.fit_transform(df['interior']).astype('uint8')


df.head()

unique values in 'color' column before cleaning 
 ['gray' 'black' 'white' 'silver' 'blue' 'brown' '—' 'beige' 'green' nan
 'red' 'gold' 'purple' 'charcoal' 'burgundy' 'turquoise' 'orange'
 'off-white' 'pink' 'yellow']


unique values in 'interior' column before cleaning 
 ['black' '—' 'gray' 'brown' 'tan' 'beige' 'off-white' 'blue' 'red' 'white'
 nan 'silver' 'burgundy' 'purple' 'gold' 'orange' 'green']

filled color with: black

filled interior with: black


,year,model,trim,body,transmission,state,condition,odometer,color,interior,seller,mmr,selling_price,sale_date
0,2014,3 Series,328i SULEV,3,0,ca,4.5,1331.0,7,1,"""financial services remarketing (lease)""",31900,30000,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
1,2014,6 Series Gran Coupe,650i,3,0,ca,4.3,2641.0,7,1,"""financial services remarketing (lease)""",66000,67000,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)
2,2014,M5,Base,3,0,ca,3.4,14943.0,1,1,"""the hertz corporation""",69000,65000,Wed Dec 17 2014 12:30:00 GMT-0800 (PST)
3,2014,6 Series,650i,0,0,ca,3.4,8819.0,1,1,"""the hertz corporation""",68000,67200,Wed Dec 17 2014 12:30:00 GMT-0800 (PST)
4,2014,5 Series,528i,3,0,ca,2.9,25969.0,1,1,"""financial services remarketing (lease)""",34200,30000,Tue Feb 03 2015 04:30:00 GMT-0800 (PST)


In [13]:
# takes the sale date column and splits from the 'GMT' and convert to proper datetime format
df['sale_date'] = pd.to_datetime(df['sale_date'].str.split(' GMT').str[0],
                                 format='%a %b %d %Y %H:%M:%S',
                                 errors='coerce')

# extract the date part with YYYY-MM-DD format
df['sale_date_only'] = df['sale_date'].dt.strftime('%Y-%m-%d')
# extract the time part with HH-MM-SS format
df['sale_time_only'] = df['sale_date'].dt.strftime('%H-%M-%S')

# drop the original date column cuz we dont need it 
df = df.drop('sale_date', axis=1)

df.head()

,year,model,trim,body,transmission,state,condition,odometer,color,interior,seller,mmr,selling_price,sale_date_only,sale_time_only
0,2014,3 Series,328i SULEV,3,0,ca,4.5,1331.0,7,1,"""financial services remarketing (lease)""",31900,30000,2015-01-15,04-30-00
1,2014,6 Series Gran Coupe,650i,3,0,ca,4.3,2641.0,7,1,"""financial services remarketing (lease)""",66000,67000,2014-12-18,12-30-00
2,2014,M5,Base,3,0,ca,3.4,14943.0,1,1,"""the hertz corporation""",69000,65000,2014-12-17,12-30-00
3,2014,6 Series,650i,0,0,ca,3.4,8819.0,1,1,"""the hertz corporation""",68000,67200,2014-12-17,12-30-00
4,2014,5 Series,528i,3,0,ca,2.9,25969.0,1,1,"""financial services remarketing (lease)""",34200,30000,2015-02-03,04-30-00


In [14]:
# extracting only the year from date column
df['sale_year'] = pd.to_datetime(df['sale_date_only']).dt.year

df['car_age'] = df['sale_year'] - df['year']

df['price_difference'] = df['selling_price'] - df['mmr']

df.head()

,year,model,trim,body,transmission,state,condition,odometer,color,interior,seller,mmr,selling_price,sale_date_only,sale_time_only,sale_year,car_age,price_difference
0,2014,3 Series,328i SULEV,3,0,ca,4.5,1331.0,7,1,"""financial services remarketing (lease)""",31900,30000,2015-01-15,04-30-00,2015,1,-1900
1,2014,6 Series Gran Coupe,650i,3,0,ca,4.3,2641.0,7,1,"""financial services remarketing (lease)""",66000,67000,2014-12-18,12-30-00,2014,0,1000
2,2014,M5,Base,3,0,ca,3.4,14943.0,1,1,"""the hertz corporation""",69000,65000,2014-12-17,12-30-00,2014,0,-4000
3,2014,6 Series,650i,0,0,ca,3.4,8819.0,1,1,"""the hertz corporation""",68000,67200,2014-12-17,12-30-00,2014,0,-800
4,2014,5 Series,528i,3,0,ca,2.9,25969.0,1,1,"""financial services remarketing (lease)""",34200,30000,2015-02-03,04-30-00,2015,1,-4200


In [15]:
# cleaned - with no NaN values
df.isnull().sum()

year                0
model               0
trim                0
body                0
transmission        0
state               0
condition           0
odometer            0
color               0
interior            0
seller              0
mmr                 0
selling_price       0
sale_date_only      0
sale_time_only      0
sale_year           0
car_age             0
price_difference    0
dtype: int64

In [16]:
# saving cleaned data
# using pickle format to preserve data types
df.to_pickle('Cleaned_Bmw_Data.pkl')